Esto no está ni medianamente bien hecho

In [2]:
from osgeo import gdal, gdalconst
import math
import numpy as np
import subprocess

In [12]:
def determine_data_type(data):
    min_val = np.nanmin(data)
    max_val = np.nanmax(data)
    print(min_val)
    print(max_val)
    
    if np.issubdtype(data.dtype, np.integer):
        if min_val >= 0:
            if max_val <= 255:
                return np.uint8
            elif max_val <= 65535:
                return np.uint16
            else:
                return np.uint32
        else:
            if min_val >= -128 and max_val <= 127:
                return np.int8
            elif min_val >= -32768 and max_val <= 32767:
                return np.int16
            else:
                return np.int32
    else:
        if min_val >= -3.4e38 and max_val <= 3.4e38:
            return np.float32
        else:
            return np.float64

In [15]:
"""Change the number of decimals"""
"""This shit does not take space for huge data with decimals that requires float32"""
# Open the raster file
raster_file = r"Z:\z_resources\im-nca-germany\results_shared_with_destatis\german_terrestrial_ecosystem_type_results_epsg25832\german_terrestrial_ecosystem_type.tiff"
output_file = r"Z:\z_resources\im-nca-germany\results_shared_with_destatis\german_terrestrial_ecosystem_type_results_epsg25832\german_terrestrial_ecosystem_type_1.tiff"
input_raster = gdal.Open(raster_file, gdal.GA_ReadOnly)

# Number of decimal places to round to (It does not work)
decimal_places = 4

new_nodata_value = -250  # Replace with your desired NoData value

# Read the first band to analyze data type
raster_band = input_raster.GetRasterBand(1)
# Esto hay que mejorarlo, quiero que tome todo, no solo 
first_scanline = raster_band.ReadAsArray(0, 0, raster_band.XSize, 1)

# Determine the optimal data type based on the first scanline
optimal_dtype = determine_data_type(first_scanline)

# Create an output raster file with the determined data type
driver = gdal.GetDriverByName('GTiff')
output_raster = driver.Create(
    output_file, 
    input_raster.RasterXSize, 
    input_raster.RasterYSize, 
    input_raster.RasterCount, 
    gdalconst.GDT_Float32 if np.issubdtype(optimal_dtype, np.floating) else gdalconst.GDT_Int32,
    options = ['COMPRESS=DEFLATE', 'TILED=YES']
)

# Copy over the geotransform and projection from the input raster
output_raster.SetGeoTransform(input_raster.GetGeoTransform())
output_raster.SetProjection(input_raster.GetProjection())

# Loop through each band in the raster
for band_num in range(1, input_raster.RasterCount + 1):
    input_band = input_raster.GetRasterBand(band_num)
    output_band = output_raster.GetRasterBand(band_num)
    
    # Set the new NoData value
    output_band.SetNoDataValue(new_nodata_value)
    
    for y in range(input_band.YSize):
        scanline = input_band.ReadAsArray(0, y, input_band.XSize, 1).astype(np.float32)
        
        # Round the values to the specified number of decimal places
        rounded_scanline = np.round(scanline, decimals=decimal_places)
        
        # Replace NoData values in the rounded_scanline with the new NoData value
        mask = np.isnan(scanline)
        rounded_scanline[mask] = new_nodata_value
        
        # Convert rounded_scanline to the optimal data type before writing
        rounded_scanline = rounded_scanline.astype(optimal_dtype)
        
        # Write the modified scanline to the output raster
        output_band.WriteArray(rounded_scanline, 0, y)

# Close the input and output raster files
input_raster = None
output_raster = None

37.1478286657143
46.938502811428584


In [11]:
optimal_dtype

numpy.float32